In [1]:
import glob
import os
import numpy as np
import time as systime
import re 
import textgrid as tg

from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from ipyparallel import Client
import _pickle as pickle

import sys

- we need to start with a top level string 
- do some edit operations to get bottom level
- have each bottom level type index a 3-state hmm
- for each 3 state hmm
	- sample some transitions etc 
	- have each state index a GMM
	- each GMM generate a 2-dimensional vector 
    
- input: top level string
- parameters we need to specify: 
	- edit operation parameters
		- ins_top (1)
		- ins_bot (n)
		- sub (n) 
	- hmm 
		- initial (3)
		- transition (3x3)
		- emission (3xm)
    - GMM
		- num of components 
		- mean, variance for each component 
		- mixture proportions 

for simplicity, let's make all top level strings 10 characters long.
let's have a 4 character alphabet 

In [2]:
top_alphabet = [0,1,2,3]

 let's have 1 more bottom level PLU



In [3]:
bot_alphabet = [0,1,2,3,4]

Now we get some random sequences of top letters that represent a top alphabet

In [4]:
top_strings = np.random.choice(top_alphabet, (100, 10))

Now we define prob distributions over ins, sub, del, and sample edit operations for each top-level PLU

In [9]:
# Dirichlet parameters for distribution over edit ops for each of the top PLUs 
# (plus distribution over final insert bottoms)
# (insert_top prob) (insert_bottom probs) (sub probs)
ops_priors = [
    [1,1,1,1,1,1,3,2,1,1,1],
    [2,1,1,1,1,1,1,3,2,1,1],
    [3,1,1,1,1,1,1,1,3,1,1],
    [1,1,1,2,2]
]

# Draw a sample from each Dirichlet to get the distribution
ops_dists = [np.random.dirichlet(i) for i in ops_priors]

# ins_top = [2]
# ins_bot = [.5,1,.5,4,2]
# sub = [[100, 3, 4, 1],
#       [.5, 100, 2, 2],
#       [2, .75, 100, 3],
#       [3,2,1,100]]


# full_dir = np.array(ins_top + ins_bot + sub[0] + sub[1] + sub[2] + sub[3], dtype=np.float64)
# # normalize 
# full_dir = full_dir/np.sum(full_dir)
# print(full_dir)


In [8]:
bottom_strings = []
i = 0
while i < len(top_strings):
    ts = top_strings[i]
    j = 0
    bs = []
    while j < len(ts):
#         sample some edit operation
#         eo = np.argmax(np.random.multinomial(1, full_dir))
        eo = np.argmax(np.random.multinomial(1, ops_dists[ts[j]]))
        if eo == 0:
#             insert top, do nothing
            j+=1
        elif eo >= 1 and eo <= 5:
            bc = bot_alphabet[eo-1]
            bs.append(bc)
            continue
        else:
#             sub
#             bc = bot_alphabet[(eo-5)%4] 
            bc = bot_alphabet[(eo-5)] 
            bs.append(bc)
            j+=1
    i+=1
    bottom_strings.append(bs)
    print(bottom_strings)


[[1, 4, 4, 2, 3, 2, 2, 2, 4, 2, 2, 3, 1, 2, 2, 4, 2, 4, 4, 4, 4, 3, 2, 2, 2, 4, 3]]
[[1, 4, 4, 2, 3, 2, 2, 2, 4, 2, 2, 3, 1, 2, 2, 4, 2, 4, 4, 4, 4, 3, 2, 2, 2, 4, 3], [3, 0, 1, 1, 1, 2, 2, 0, 0, 1, 2, 2, 1, 1, 3, 2, 0]]
[[1, 4, 4, 2, 3, 2, 2, 2, 4, 2, 2, 3, 1, 2, 2, 4, 2, 4, 4, 4, 4, 3, 2, 2, 2, 4, 3], [3, 0, 1, 1, 1, 2, 2, 0, 0, 1, 2, 2, 1, 1, 3, 2, 0], [2, 0, 2, 1, 1, 0, 1, 4, 2, 4, 1, 2, 2, 4, 1, 3, 2, 4, 4, 4, 4, 4, 4, 4, 0, 2, 4, 4, 4, 3, 4, 3]]
[[1, 4, 4, 2, 3, 2, 2, 2, 4, 2, 2, 3, 1, 2, 2, 4, 2, 4, 4, 4, 4, 3, 2, 2, 2, 4, 3], [3, 0, 1, 1, 1, 2, 2, 0, 0, 1, 2, 2, 1, 1, 3, 2, 0], [2, 0, 2, 1, 1, 0, 1, 4, 2, 4, 1, 2, 2, 4, 1, 3, 2, 4, 4, 4, 4, 4, 4, 4, 0, 2, 4, 4, 4, 3, 4, 3], [1, 3, 1, 1, 3, 1, 3, 1, 0, 1, 2, 2, 0, 0, 2, 1, 4, 3, 2, 3, 4, 3]]
[[1, 4, 4, 2, 3, 2, 2, 2, 4, 2, 2, 3, 1, 2, 2, 4, 2, 4, 4, 4, 4, 3, 2, 2, 2, 4, 3], [3, 0, 1, 1, 1, 2, 2, 0, 0, 1, 2, 2, 1, 1, 3, 2, 0], [2, 0, 2, 1, 1, 0, 1, 4, 2, 4, 1, 2, 2, 4, 1, 3, 2, 4, 4, 4, 4, 4, 4, 4, 0, 2, 4, 4, 4, 3, 4, 3], [1, 3,

Let's define the HMM/GMM parameters

In [28]:
# transition = [[.5, .5, 0],
#              [0, .5, .5],
#              [.75, 0, .25]]

components = []
# Distribution over components for each bottom PLU state
components.append([[.7,.3],
             [.9,.1],
             [.3,.7]])
components.append([[.3,.7],
             [.5,.5],
             [.8,.2]])
components.append([[.4,.6],
             [.2,.8],
             [.9,.1]])
components.append([[.5,.5],
             [.3,.7],
             [.6,.4]])
components.append([[.1,.9],
             [.6,.4],
             [.7,.3]])


# We need a set of parameters for each component
# for each HMM state for each bottom PLU
# So in this case, we need 2 * 3 * 5 = 30 sets of parameters
component_parameters = [
    [
        [
            [[0,0], [[1,0],[0,1]]], # bottom PLU 0, HMM state 0, component 0
            [[1,1], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 0, component 1
        ],
        [
             [[2,2], [[1,0],[0,1]]], # bottom PLU 0, HMM state 1, component 0
             [[3,3], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 1, component 1
        ],
        [
            [[4,4], [[1,0],[0,1]]], # bottom PLU 0, HMM state 2, component 0
            [[5,5], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 2, component 1
        ]
    ],
    [
        [
            [[10,10], [[1,0],[0,1]]], # bottom PLU 1, HMM state 0, component 0
            [[11,11], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 0, component 1
        ],
        [
             [[12,12], [[1,0],[0,1]]], # bottom PLU 1, HMM state 1, component 0
             [[13,13], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 1, component 1
        ],
        [
            [[14,14], [[1,0],[0,1]]], # bottom PLU 1, HMM state 2, component 0
            [[15,15], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 2, component 1
        ]
    ],
    [        
        [
            [[20,20], [[1,0],[0,1]]], # bottom PLU 2, HMM state 0, component 0
            [[21,21], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 0, component 1
        ],
        [
             [[22,22], [[1,0],[0,1]]], # bottom PLU 2, HMM state 1, component 0
             [[23,23], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 1, component 1
        ],
        [
            [[24,24], [[1,0],[0,1]]], # bottom PLU 2, HMM state 2, component 0
            [[25,25], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 2, component 1
        ]
    ],
    [
        [
            [[30,30], [[1,0],[0,1]]], # bottom PLU 3, HMM state 0, component 0
            [[31,31], [[1,0],[0,1]]]  # bottom PLU 3, HMM state 0, component 1
        ],
        [
             [[32,32], [[1,0],[0,1]]], # bottom PLU 3, HMM state 1, component 0
             [[33,33], [[1,0],[0,1]]]  # bottom PLU 3, HMM state 1, component 1
        ],
        [
            [[34,34], [[1,0],[0,1]]], # bottom PLU 3, HMM state 2, component 0
            [[35,35], [[1,0],[0,1]]]  # bottom PLU 3, HMM state 2, component 1
        ]
    ],
    [
        [
            [[40,40], [[1,0],[0,1]]], # bottom PLU 4, HMM state 0, component 0
            [[41,41], [[1,0],[0,1]]]  # bottom PLU 4, HMM state 0, component 1
        ],
        [
             [[42,42], [[1,0],[0,1]]], # bottom PLU 4, HMM state 1, component 0
             [[43,43], [[1,0],[0,1]]]  # bottom PLU 4, HMM state 1, component 1
        ],
        [
            [[44,44], [[1,0],[0,1]]], # bottom PLU 4, HMM state 2, component 0
            [[45,45], [[1,0],[0,1]]]  # bottom PLU 4, HMM state 2, component 1
        ]
    ]


]
# n1 = [[0,0], [[1,0],[0,1]]]
# n2 = [[10, 10], [[1,0],[0,1]]]


array([0, 1, 0])

In [29]:
def sample_HMMGMM(bs):
    mfccs = []
    for i, char in enumerate(bs):
        vec = None
#         if i%10 == 0:
#             print("done with ", i)
#         start_state = np.random.choice([0,1,2], p=init)
        curr_state = 0
#       get the mfcc vector and transition
        while curr_state < 3:
            gmm_component_choice = np.random.choice(2,p=components[char][curr_state])
            vec = np.random.multivariate_normal(*component_parameters[char][curr_state][gmm_component_choice])
#             vec = components[curr_state][0]*np.random.multivariate_normal(*n1) + \
#                     components[curr_state][1]*np.random.multivariate_normal(*n2)
            
            mfccs.append(vec)
#             new_curr_state = np.random.choice([0,1,2], p=transition[curr_state])
            # Transition to next state with 50% probability
            curr_state = np.random.choice([curr_state,curr_state+1],p=[0.5,0.5])
#             if curr_state == 2 and new_curr_state == 0:
    #         get the mfcc vector and transition
#             curr_state = new_curr_state
    return np.array(mfccs)

all_data = []
for i, bs in enumerate(bottom_strings):
    mfccs = sample_HMMGMM(bs)
    all_data.append(mfccs)

all_data = np.array(all_data)
print(all_data[0].shape)
print(all_data.shape)
        
# for i,line in enumerate(all_data):
#     with open("/Users/esteng/ULD/audio/example_2d/sent_{}.fea", "w") as f1:
        
        

(145, 2)
(100,)


In [9]:
import amdtk
data = amdtk.read_htk("/Users/esteng/ULD/audio/icicles/icicles.fea")
print(data.shape)

INSIDE INIT
SUCCESSFUL IMPORT IN INIT
(285, 39)


In [10]:
def collect_data_stats(data):
    """Job to collect the statistics."""
    # We  re-import this module here because this code will run
    # remotely.
    
    stats_0 = data.shape[0]
    stats_1 = data.sum(axis=0)
    stats_2 = (data**2).sum(axis=0)
    retval = (
        stats_0,
        stats_1,
        stats_2
    )
    return retval

data_stats = list(map(collect_data_stats, all_data))



In [11]:

def accumulate_stats(data_stats):
    n_frames = data_stats[0][0]
    mean = data_stats[0][1]
    var = data_stats[0][2]
    for stats_0, stats_1, stats_2 in data_stats[1:]:
        n_frames += stats_0
        mean += stats_1
        var += stats_2
    mean /= n_frames
    var = (var / n_frames) - mean**2

    data_stats = {
        'count': n_frames,
        'mean': mean,
        'var': var
    }
    return data_stats

final_data_stats = accumulate_stats(data_stats)


In [12]:
import subprocess
num_tops = 4

print("starting engines")
subprocess.Popen(['ipcluster', 'start',' --profile', 'default',' -n', '4', '--daemonize'])
subprocess.Popen(['sleep', '10']).communicate()


rc = Client(profile='default')
dview = rc[:]
print('Connected to', len(dview), 'jobs.')


elbo = []
time = []
def callback(args):
    elbo.append(args['objective'])
    time.append(args['time'])
    print('elbo=' + str(elbo[-1]), 'time=' + str(time[-1]))
 

print("Creating phone loop model...")
conc = 0.1



model = amdtk.PhoneLoopNoisyChannel.create(
    n_units=5,  # number of acoustic units
    n_states=3,   # number of states per unit
    n_comp_per_state=3,   # number of Gaussians per emission
    n_top_units=num_tops, # size of top PLU alphabet
    mean=np.zeros_like(final_data_stats['mean']), 
    var=np.ones_like(final_data_stats['var']),
    max_slip_factor=.05
    #concentration=conc
)


print("Creating VB optimizer...")   
optimizer = amdtk.ToyNoisyChannelOptimizer(
    dview, 
    final_data_stats, 
    args= {'epochs': 4,
     'batch_size': 1,
     'lrate': 0.01,
     'pkl_path': "example_test/",
     'log_dir': 'logs'},
    model=model,

)

print("Running VB optimization...")
begin = systime.time()
print("running with {} paths".format(len(list(zipped_paths))))
optimizer.run(zipped_paths, callback)
end = systime.time()
print("VB optimization took ",end-begin," seconds.")

# fig1 = figure(
#     x_axis_label='time (s)', 
#     y_axis_label='ELBO',
#     width=400, 
#     height=400
# )
# x = np.arange(0, len(elbo), 1)
# fig1.line(x, elbo)
#show(fig1)

print("\nDECODING\n")

date_string = systime.strftime("textgrids_%Y-%m-%d_%H:%M")

# Need to change this according to 
samples_per_sec = 100


for (fea_path, top_path) in zipped_paths:



    data = amdtk.read_htk(fea_path)

    # Normalize the data
    data_mean = np.mean(data)
    data_var = np.var(data)
    data = (data-data_mean)/np.sqrt(data_var)

    # Read top PLU sequence from file
    with open(top_path, 'r') as f:
        topstring = f.read()
        tops = topstring.strip().split(',')
        tops = [int(x) for x in tops]

    #result = model.decode(data, tops, state_path=False)
    #result_path = model.decode(data, tops, state_path=True)
    (result_intervals, edit_path) = model.decode(data, tops, phone_intervals=True, edit_ops=True)

    print("---")
    print("Phone sequence for file", fea_path, ":")
    print(result_intervals)
    print(edit_path)
    

starting engines
Connected to 4 jobs.
Creating phone loop model...
Creating VB optimizer...
importing numpy on engine(s)
importing read_htk from amdtk on engine(s)
importing _pickle on engine(s)
importing os on engine(s)
Running VB optimization...


NameError: name 'zipped_paths' is not defined